<a href="https://colab.research.google.com/github/AnggitaGayatri/Copywriting-Otomatis-Produk-Fashion-Menggunakan-T5/blob/Anggita-dev/%20mT5-Indonesia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Required Libraries

In [ ]:
!pip install datasets transformers evaluate -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=88ca92984684919684d36d89a224482f4362d4b86f4fbed6229b5ccef2ac7939
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


# Import Required Libraries

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import DataCollatorForSeq2Seq
from transformers import pipeline
import evaluate
import os
import warnings
import random


os.environ["WANDB_DISABLED"] = "true"
warnings.filterwarnings("ignore")

# Load Dataset

In [ ]:
data = pd.read_csv("data_matahari_tfidf.csv")
data['deskripsi'] = data['deskripsi'].astype(str)
data['keywords'] = data['keywords'].astype(str)


dataset = Dataset.from_pandas(data)
dataset = dataset.train_test_split(test_size=0.2)
dataset["validation"] = dataset["test"].train_test_split(test_size=0.5)["test"]

print(dataset)


DatasetDict({
    train: Dataset({
        features: ['deskripsi', 'keywords'],
        num_rows: 5060
    })
    test: Dataset({
        features: ['deskripsi', 'keywords'],
        num_rows: 1266
    })
    validation: Dataset({
        features: ['deskripsi', 'keywords'],
        num_rows: 633
    })
})


simpan hasil pembagian dataset

In [ ]:
# Konversi train set ke Pandas DataFrame
train_data = dataset["train"].to_pandas()
test_data = dataset["test"].to_pandas()

# Simpan DataFrame ke file CSV
train_data.to_csv("train_data.csv", index=False)
test_data.to_csv("test_data.csv", index=False)

print("Data train berhasil disimpan ke file 'train_data.csv'")


Data train berhasil disimpan ke file 'train_data.csv'


# Preprocessing Data

In [ ]:
model_checkpoint = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

max_input_length = 64
max_target_length = 256

# Preprocess function
def preprocess_function(data):
    inputs = data["keywords"]
    targets = data["deskripsi"]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    labels = tokenizer(targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenize dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/5060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1266 [00:00<?, ? examples/s]

Map:   0%|          | 0/633 [00:00<?, ? examples/s]

# Fine-Tuning Model

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

# Training arguments
batch_size = 8
args = Seq2SeqTrainingArguments(
    output_dir="./mT5-text-generation",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=20,
    predict_with_generate=True,
    logging_steps=10,
)


data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

# Train model
trainer.train()
trainer.save_model("./mT5-text-generation")


pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,2.734500,1.968958
2,1.876700,1.466454
3,1.807000,1.220065
4,1.626000,1.071231
5,1.381100,0.965512
6,1.288600,0.888932
7,1.242500,0.839360
8,0.977900,0.790402
9,1.117100,0.748585
10,1.102400,0.722785


model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

# Simpan Model

In [ ]:
save_directory = "./mT5-finetuned-model-fix"

os.makedirs(save_directory, exist_ok=True)
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

print(f"Model dan tokenizer telah disimpan di folder: {save_directory}")

In [ ]:
zip_filename = "mT5-finetuned-model-fix.zip"
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk(save_directory):
        for file in files:
            zipf.write(os.path.join(root, file),
                       os.path.relpath(os.path.join(root, file), save_directory))

print(f"Model telah dikompres menjadi {zip_filename}")

# Unduh file ZIP
from google.colab import files
files.download(zip_filename)

# Generasi Teks

In [ ]:
# Load fine-tuned model
text_generator = pipeline("text2text-generation", model="./mT5-text-generation", tokenizer=tokenizer)

# Contoh penggunaan
keywords = "sneakers, nevada, nyaman"
generated_description = text_generator(keywords, max_length=256)[0]["generated_text"]

print(f"Keywords: {keywords}")
print(f"Generated Description: {generated_description}")


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Keywords: sneakers, nevada, nyaman
Generated Description: Material sintetis Sneakers sneakers wanita Detail sneakers nyaman Detail sneakers nyaman dipakai reguler pas Cocok untuk gaya kasual Sepatu sneakers wanita dengan desain sneakers untuk menunjang penampilan Anda menjadi lebih menawan sehari-hari. Model yang simpel dan bahan yang nyaman digunakan, akan nyaman saat digunakan sepanjang hari.


In [ ]:
keywords = "sepatu, kulit, pria, putih"
generated_description = text_generator(keywords, max_length=256)[0]["generated_text"]

In [ ]:
print(f"Keywords: {keywords}")
print(f"Generated Description: {generated_description}")

Keywords: sepatu, kulit, pria, warna, putih
Generated Description: Material kulit katun Sepatu pria Detail warna nyaman Model sepatu pria Detail warna yang menawan Detail warna yang menawan Detail warna yang menawan Detail warna yang menawan Detail warna yang menawan Detail warna yang menawan Detail warna yang menawan Detail warna yang menawan Detail warna yang menawan Detail warna yang menawan Detail warna yang menawan Detail warna yang menawan Detail warna yang menawan Detail warna yang menawan Detail warna yang menawan Detail warna yang menawan Detail warna yang menawan Detail warna yang menawan Detail warna yang menawan Detail warna yang menawan Detail warna yang menawan Detail warna dan warna yang menawan Detail warna yang menawan Detail warna yang menawan Detail warna yang menawan Detail warna yang menawan Detail warna yang menawan Detail warna yang menawan Detail warna yang menawan Detail warna yang menawan Detail warna yang menawan Detail warna yang menawan Detail warna yang me

# Evaluasi

load data testing

In [ ]:
# Muat data CSV
data = pd.read_csv("test_data.csv")
data['deskripsi'] = data['deskripsi'].astype(str)
data['keywords'] = data['keywords'].astype(str)

# Konversi ke Hugging Face Dataset
dataset = Dataset.from_pandas(data)

print(dataset)


Dataset({
    features: ['deskripsi', 'keywords'],
    num_rows: 1266
})


load model yang telah di training

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model_dir = "/content/drive/My Drive/deep learning/DL - MODELING/FIX/Model/mT5-finetuned-model-fix"

In [ ]:
# Load model dan tokenizer dari Google Drive
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(model_dir)

print("Model dan tokenizer berhasil dimuat.")


You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


Model dan tokenizer berhasil dimuat.


In [ ]:
def text_generator(keywords, max_length=256):
    # Tokenisasi input
    inputs = tokenizer(
        keywords,
        return_tensors="pt",
        truncation=True,
        padding=True
    )
    inputs = {k: v.to(model.device) for k, v in inputs.items()}  # Pindahkan ke perangkat model

    # Generate text
    generated_ids = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=max_length,
        num_beams=5,  # Beam search
        repetition_penalty=2.5,
        early_stopping=True
    )

    # Decode hasil prediksi
    generated_text = tokenizer.decode(
        generated_ids[0],
        skip_special_tokens=True,
        clean_up_tokenization_spaces=True
    )
    return [{"generated_text": generated_text}]

In [ ]:
rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")


sample_keywords = dataset["keywords"]
sample_references = dataset["deskripsi"]
sample_predictions = [text_generator(kw, max_length=256)[0]["generated_text"] for kw in sample_keywords]

rouge_results = rouge.compute(predictions=sample_predictions, references=sample_references)
bleu_results = bleu.compute(predictions=sample_predictions, references=[[ref] for ref in sample_references])

print("ROUGE Results (full dataset):", rouge_results)
print("BLEU Results (full dataset):", bleu_results)


# Save Prediction

In [ ]:
results_df = pd.DataFrame({
    "keywords": dataset["test"]["keywords"],
    "deskripsi_asli": dataset["test"]["deskripsi"],
    "hasil_prediksi": predictions
})


results_df.to_csv("hasil_prediksi.csv", index=False)

NameError: name 'predictions' is not defined